In [ ]:
import pandas as pd

import plotly.offline as offline
import plotly.graph_objs as go
import cufflinks as cf
cf.go_offline()

offline.init_notebook_mode()

# Read movie data

In [ ]:
movie_filename = "../data/movies.csv"

In [ ]:
movies_df = pd.read_csv(movie_filename)
print(movies_df.shape)
count_movie_ids = movies_df.shape[0]
movies_df.head(5)

In [ ]:
movies_df = movies_df.set_index('movieId')
movies_df.sample(5)

sample lets us grab a few random rows in our data
- https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sample.html

## Add some useful columns

In [ ]:
## Turn the genres into a list
movies_df['genres_list'] = movies_df['genres'].apply(lambda x: x.split('|'))
movies_df.sample(5)

we just used the apply function to generate information from a column
- https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.apply.html

In [ ]:
## Make a column with the 'Year' extracted from the title
## use a function bc why not

def extract_year_from_title(x):
    '''
    Title has format --Title--(YYYY)
    Title could contain () so must only take the last one
    Should expect to turn YYYY into int
    '''
    *title, year = x.split('(')
    year = year.replace(')','')
    try:
        year = int(year)
    except:
        year = 'unknown'
    return year

movies_df['year'] = movies_df['title'].apply(extract_year_from_title)

In [ ]:
## find some movies with no year listed ... (according to our function)
movies_df.loc[movies_df['year']=='unknown']

we just used loc to find particular data
- https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.loc.html

In [ ]:
## drop any movies for which I do not have a year
movies_df = movies_df.loc[movies_df['year']!='unknown']
print("Left with {} movies, dropped {} movies without a year".format(movies_df.shape[0],
                                                      count_movie_ids - movies_df.shape[0]))
## update count_movie_ids
count_movie_ids = movies_df.shape[0]

In [ ]:
## gather the movies by year and see how many there are each year
no_movies_by_year = movies_df.groupby('year').count()

print(no_movies_by_year.head(5))
## plot it
fig = no_movies_by_year[['title']].iplot(kind='scatter',
                                         title = 'Movies by Year',
                                         xTitle = 'Year',
                                         yTitle = 'Count',
                                         asFigure=True, )

offline.plot(fig, filename='../graphs/movies_by_year.html', )

**Whoa!** 

We just used groupby
- https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html

Then we aggregated stuff using count()
- https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.count.html

But more generally we could have used an agg function
- https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.agg.html

Then, we plotted using Plotly and cufflinks 
- https://plot.ly/ipython-notebooks/cufflinks/
- https://github.com/santosjorge/cufflinks

In [ ]:
## lets explore the yearly trends by genre

In [ ]:
## turn the genres_list into a row with the index as the column
genres_df = movies_df['genres_list'].apply(pd.Series)
genres_df.head(3)

In [ ]:
## stack those values to make more sense of it
genres_df = genres_df.stack(level=-1)
print(type(genres_df))
genres_df.head(10)

we've used stack to stack the columns and the values
- https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.stack.html

In [ ]:
## get it back as a dataframe so that you can have fun with it
genres_df = genres_df.reset_index()
genres_df.head(3)

In [ ]:
## pivot those values to make them column names
genres_df['is_genre'] = True
genres_df = genres_df.pivot(index='movieId',columns=0,values='is_genre')
print("Generated {} movies, should have {}, discrepancy --> {}".format(genres_df.shape[0], 
                                                                       count_movie_ids, 
                                                                       count_movie_ids - genres_df.shape[0]))
genres_df.head(3)

pivot is just like what the name suggests, it pivots row values and makes them column names
- https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.pivot.html

In [ ]:
## fill the empty values 
genres_df = genres_df.fillna(False)
genres_df.head(3)
# genres.loc[genres['(no genres listed)']==True]

In [ ]:
## make a bar chart for each genre mentioned
genres_df.sum().iplot(kind='bar',
                      yTitle = 'Non-unique Count')

In [ ]:
## merge the data 
movies_with_genre_df = movies_df[['title','year']].merge(genres_df, 
                                  left_index=True, 
                                  right_index=True)

print("{} movie ids, originally had {}, lost {}".format(movies_with_genre_df.shape[0],
                                                       count_movie_ids,
                                                       count_movie_ids - movies_with_genre_df.shape[0]))

merge lets us bring together two dataframes, it has heaps of different parameters you can select
- https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html
In general to bring together dataframes you can use concat, merge or join, 
- https://pandas.pydata.org/pandas-docs/stable/merging.html

In [ ]:
## plot the differen genres by year
movies_with_genre_df.groupby('year').sum().iplot()